In [3]:

import saopaulo.cycling_potential as cp

print(cp.partial_cycling_potential('distance', 'm', 4820))

0.33785064661258535


In [ ]:
# importing data and auxiliar functions

import saopaulo.sp_grid as gr
import saopaulo.cycling_potential as cp
from bikescience.slope import plot_slope, plot_slopes, split_route
from shapely.geometry import LineString
import ast

from math import floor

import folium
import pandas as pd
import geopandas as gpd
from statistics import stdev
from ipywidgets import interact_manual, widgets, fixed
from IPython.core.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import warnings
import requests
import os
warnings.simplefilter('ignore')
import time

zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
zone_shp.crs = {'init': 'epsg:31983'}  
zone_shp.to_crs(epsg='4326', inplace=True)

routes_files = {
    'pedestrian':  'routes_pedestrians_to_bikes.csv',
    'car':         'new_routes_cars_to_bikes.csv',
    'bus_inter':   'new_routes_bus_intercity_to_bikes.csv',
    'bus_otherc':  'new_routes_bus_other_cities_to_bikes.csv',
    'bus_private': 'new_routes_bus_private_to_bikes.csv',
    'bus_school':  'new_routes_bus_school_to_bikes.csv',
    'bus_sp':      'new_routes_bus_sp_to_bikes.csv',
    'monorail':    'new_routes_monorail_to_bikes.csv',
    'motorcycle':  'new_routes_motorcycle_to_bikes.csv',
    'other':       'new_routes_other_to_bikes.csv',
    'subway':      'new_routes_subway_to_bikes.csv',
    'taxi_app':    'new_routes_taxi_app_to_bikes.csv',
    'taxi':        'new_routes_taxi_to_bikes.csv',
    'train':       'new_routes_train_to_bikes.csv',
}
    
# read routes files and join with general trips file
od_trips = pd.read_csv('../data/sao-paulo/od/OD_2017.csv')
od_trips.columns = [c[: c.index(',')] for c in od_trips.columns]
od_trips.set_index('ID_ORDEM', inplace=True, drop=False)
display(od_trips)

complete_gdf = gpd.GeoDataFrame()

for modal in routes_files:
    print('importando' + modal)
    df = pd.read_csv('../data/sao-paulo/rotas-migracao/' + routes_files[modal])
    df.set_index('trip_id', inplace=True, drop=False)
    df = df.join(other = od_trips, how = 'left')
    gdf = gpd.GeoDataFrame(df, crs={'init': 'epsg:4326'}, 
                           geometry=[LineString(ast.literal_eval(r)) for r in df.route])
    gdf['length'] = gdf['geometry'].length * 100000
    gdf['modal'] = modal
    complete_gdf = pd.concat([complete_gdf, gdf])


od_routes_sp = gpd.read_file('../data/sao-paulo/od/routes/bike_routes_in_sp.shp')
od_routes_sp.crs = {'init': 'epsg:4326'}
od_routes_sp['length'] = od_routes_sp['geometry'].length * 100000

the_grid = gr.create(n=10, west_offset=-0.15, east_offset=0.23, north_offset=0.19, south_offset=-0.46)

/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


,ZONA,MUNI_DOM,CO_DOM_X,CO_DOM_Y,ID_DOM,F_DOM,FE_DOM,DOM,CD_ENTRE,DATA,...,DURACAO,MODOPRIN,TIPVG,PAG_VIAG,TP_ESAUTO,VL_EST,PE_BICI,VIA_BICI,TP_ESTBICI,ID_ORDEM
ID_ORDEM,,,,,,,,,,,,,,,,,,,,,
1,1,36,333743,7394463,10001,1,15.416667,1,1,6092017,...,10,16,3,0,0,0.0,1,0,0,1
2,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,10,16,3,0,0,0.0,1,0,0,2
3,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,50,1,1,2,0,0.0,0,0,0,3
4,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,60,1,1,2,0,0.0,0,0,0,4
5,1,36,333743,7394463,10001,0,15.416667,1,1,6092017,...,40,12,2,0,0,0.0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183088,517,25,294618,7410518,5171418,0,85.531250,1418,1,15092018,...,30,16,3,0,0,0.0,1,0,0,183088
183089,517,25,295243,7411456,5171599,1,52.500000,1599,1,15092018,...,10,16,3,0,0,0.0,1,0,0,183089
183090,517,25,295243,7411456,5171599,0,52.500000,1599,1,15092018,...,10,16,3,0,0,0.0,1,0,0,183090


importandopedestrian
importandocar


In [2]:
complete_gdf = complete_gdf.sort_index()
display(complete_gdf.head(10))

,trip_id,orig_lat,orig_lon,dest_lat,dest_lon,route,distance,num_trips,ZONA,MUNI_DOM,...,PAG_VIAG,TP_ESAUTO,VL_EST,PE_BICI,VIA_BICI,TP_ESTBICI,ID_ORDEM,geometry,length,modal
1.0,1.0,-23.551678,-46.628858,-23.551495,-46.635115,"[[-46.628827, -23.55186, 748.52], [-46.627845,...",1.973649,22.132647,1.0,36.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,"LINESTRING Z (-46.62883 -23.55186 748.52000, -...",1849.286293,pedestrian
2.0,2.0,-23.551495,-46.635115,-23.551678,-46.628858,"[[-46.635147, -23.551546, 783.14], [-46.635381...",1.847881,22.132647,1.0,36.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,"LINESTRING Z (-46.63515 -23.55155 783.14000, -...",1760.427711,pedestrian
3.0,3.0,-23.551678,-46.628858,-23.571829,-46.690238,"[[-46.628827, -23.55186, 748.52], [-46.627845,...",10.045467,18.882487,1.0,36.0,...,2.0,0.0,0.0,0.0,0.0,0.0,3.0,"LINESTRING Z (-46.62883 -23.55186 748.52000, -...",9473.535456,subway
4.0,4.0,-23.571829,-46.690238,-23.525949,-46.666559,"[[-46.690264, -23.571842, 742.26], [-46.691246...",6.921102,18.882487,1.0,36.0,...,2.0,0.0,0.0,0.0,0.0,0.0,4.0,"LINESTRING Z (-46.69026 -23.57184 742.26000, -...",6428.546731,subway
4.0,4.0,-23.525949,-46.666559,-23.537903,-46.670921,"[[-46.666835, -23.525929, 733.11], [-46.666934...",2.850943,18.882487,1.0,36.0,...,2.0,0.0,0.0,0.0,0.0,0.0,4.0,"LINESTRING Z (-46.66683 -23.52593 733.11000, -...",2667.571706,bus_sp
5.0,5.0,-23.537903,-46.670921,-23.551678,-46.628858,"[[-46.670521, -23.538166, 793.48], [-46.669261...",6.916414,18.882487,1.0,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,"LINESTRING Z (-46.67052 -23.53817 793.48000, -...",6479.024014,taxi_app
6.0,6.0,-23.552002,-46.628166,-23.551305,-46.631694,"[[-46.628153, -23.552209, 743.23], [-46.627562...",2.055874,22.132647,1.0,36.0,...,0.0,0.0,0.0,1.0,0.0,0.0,6.0,"LINESTRING Z (-46.62815 -23.55221 743.23000, -...",1923.740596,pedestrian
7.0,7.0,-23.551305,-46.631694,-23.552002,-46.628166,"[[-46.632193, -23.551043, 764.21], [-46.632568...",0.772373,22.132647,1.0,36.0,...,0.0,0.0,0.0,1.0,0.0,0.0,7.0,"LINESTRING Z (-46.63219 -23.55104 764.21000, -...",732.073056,pedestrian
8.0,8.0,-23.552002,-46.628166,-23.551305,-46.631694,"[[-46.628153, -23.552209, 743.23], [-46.627562...",2.055874,22.132647,1.0,36.0,...,0.0,0.0,0.0,1.0,0.0,0.0,8.0,"LINESTRING Z (-46.62815 -23.55221 743.23000, -...",1923.740596,pedestrian
9.0,9.0,-23.551305,-46.631694,-23.552002,-46.628166,"[[-46.632193, -23.551043, 764.21], [-46.632568...",0.772373,22.132647,1.0,36.0,...,0.0,0.0,0.0,1.0,0.0,0.0,9.0,"LINESTRING Z (-46.63219 -23.55104 764.21000, -...",732.073056,pedestrian


In [3]:
potential = []
for i, r in complete_gdf.iterrows():
    if i % 5000 == 0:
        print(i, '-------------------------')
    try:
        pot_d, pot_a, pot_i = cp.cycling_potential_variables(r)
    except:
        pot_d, pot_a, pot_i = 0,0,0
        print(i, '- broken case')
    potential.append([i, pot_d, pot_a, pot_i, (pot_d + pot_a + pot_i) / 3])
        
#display(potential)

3604.0 - broken case
3605.0 - broken case
4798.0 - broken case
4799.0 - broken case
4802.0 - broken case
4803.0 - broken case
5000.0 -------------------------
6716.0 - broken case
6717.0 - broken case
10000.0 -------------------------
15000.0 -------------------------
15000.0 -------------------------
25000.0 -------------------------
35000.0 -------------------------
40000.0 -------------------------
40000.0 -------------------------
45000.0 -------------------------
50000.0 -------------------------
58435.0 - broken case
58436.0 - broken case
58437.0 - broken case
58438.0 - broken case
60000.0 -------------------------
61341.0 - broken case
61342.0 - broken case
65000.0 -------------------------
70000.0 -------------------------
76031.0 - broken case
76032.0 - broken case
80000.0 -------------------------
85000.0 -------------------------
85194.0 - broken case
85195.0 - broken case
85196.0 - broken case
85197.0 - broken case
86627.0 - broken case
86628.0 - broken case
90000.0 -------

In [4]:
c = ['id', 'distance_potential', 'age_potential', 'inclination_potential', 'final_potential']
potential_df = pd.DataFrame(potential, columns = c)
potential_df.set_index('id', inplace=True, drop=False)
potential_df = potential_df.sort_index()

display(potential_df.head(10))

,id,distance_potential,age_potential,inclination_potential,final_potential
id,,,,,
1.0,1.0,0.650491,0.023107,0.206501,0.293366
2.0,2.0,0.689523,0.023107,0.463706,0.392112
3.0,3.0,0.032389,0.848482,0.066909,0.315927
4.0,4.0,0.077287,0.848482,0.402769,0.442846
4.0,4.0,0.391186,0.848482,0.280026,0.506565
5.0,5.0,0.076013,0.848482,0.239534,0.388010
6.0,6.0,0.619590,0.720850,0.380389,0.573610
7.0,7.0,0.797045,0.720850,0.909449,0.809115
8.0,8.0,0.619590,0.720850,0.380389,0.573610


In [5]:
complete_gdf_joined = pd.concat([complete_gdf, potential_df], axis=1)
#complete_gdf.join(other = potential_df, how = 'left')
display(complete_gdf_joined.head(10))

,trip_id,orig_lat,orig_lon,dest_lat,dest_lon,route,distance,num_trips,ZONA,MUNI_DOM,...,TP_ESTBICI,ID_ORDEM,geometry,length,modal,id,distance_potential,age_potential,inclination_potential,final_potential
1.0,1.0,-23.551678,-46.628858,-23.551495,-46.635115,"[[-46.628827, -23.55186, 748.52], [-46.627845,...",1.973649,22.132647,1.0,36.0,...,0.0,1.0,"LINESTRING Z (-46.62883 -23.55186 748.52000, -...",1849.286293,pedestrian,1.0,0.650491,0.023107,0.206501,0.293366
2.0,2.0,-23.551495,-46.635115,-23.551678,-46.628858,"[[-46.635147, -23.551546, 783.14], [-46.635381...",1.847881,22.132647,1.0,36.0,...,0.0,2.0,"LINESTRING Z (-46.63515 -23.55155 783.14000, -...",1760.427711,pedestrian,2.0,0.689523,0.023107,0.463706,0.392112
3.0,3.0,-23.551678,-46.628858,-23.571829,-46.690238,"[[-46.628827, -23.55186, 748.52], [-46.627845,...",10.045467,18.882487,1.0,36.0,...,0.0,3.0,"LINESTRING Z (-46.62883 -23.55186 748.52000, -...",9473.535456,subway,3.0,0.032389,0.848482,0.066909,0.315927
4.0,4.0,-23.571829,-46.690238,-23.525949,-46.666559,"[[-46.690264, -23.571842, 742.26], [-46.691246...",6.921102,18.882487,1.0,36.0,...,0.0,4.0,"LINESTRING Z (-46.69026 -23.57184 742.26000, -...",6428.546731,subway,4.0,0.077287,0.848482,0.402769,0.442846
4.0,4.0,-23.525949,-46.666559,-23.537903,-46.670921,"[[-46.666835, -23.525929, 733.11], [-46.666934...",2.850943,18.882487,1.0,36.0,...,0.0,4.0,"LINESTRING Z (-46.66683 -23.52593 733.11000, -...",2667.571706,bus_sp,4.0,0.391186,0.848482,0.280026,0.506565
5.0,5.0,-23.537903,-46.670921,-23.551678,-46.628858,"[[-46.670521, -23.538166, 793.48], [-46.669261...",6.916414,18.882487,1.0,36.0,...,0.0,5.0,"LINESTRING Z (-46.67052 -23.53817 793.48000, -...",6479.024014,taxi_app,5.0,0.076013,0.848482,0.239534,0.388010
6.0,6.0,-23.552002,-46.628166,-23.551305,-46.631694,"[[-46.628153, -23.552209, 743.23], [-46.627562...",2.055874,22.132647,1.0,36.0,...,0.0,6.0,"LINESTRING Z (-46.62815 -23.55221 743.23000, -...",1923.740596,pedestrian,6.0,0.619590,0.720850,0.380389,0.573610
7.0,7.0,-23.551305,-46.631694,-23.552002,-46.628166,"[[-46.632193, -23.551043, 764.21], [-46.632568...",0.772373,22.132647,1.0,36.0,...,0.0,7.0,"LINESTRING Z (-46.63219 -23.55104 764.21000, -...",732.073056,pedestrian,7.0,0.797045,0.720850,0.909449,0.809115
8.0,8.0,-23.552002,-46.628166,-23.551305,-46.631694,"[[-46.628153, -23.552209, 743.23], [-46.627562...",2.055874,22.132647,1.0,36.0,...,0.0,8.0,"LINESTRING Z (-46.62815 -23.55221 743.23000, -...",1923.740596,pedestrian,8.0,0.619590,0.720850,0.380389,0.573610
9.0,9.0,-23.551305,-46.631694,-23.552002,-46.628166,"[[-46.632193, -23.551043, 764.21], [-46.632568...",0.772373,22.132647,1.0,36.0,...,0.0,9.0,"LINESTRING Z (-46.63219 -23.55104 764.21000, -...",732.073056,pedestrian,9.0,0.797045,0.720850,0.909449,0.809115


In [ ]:
complete_gdf_joined.to_csv('complete_csv.csv')
complete_gdf_joined.to_file("countries.shp")